In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [25]:
local = "..\\dataset\\incoming\\"

In [53]:
# leitura dos datasets
customers = pd.read_csv(local+"olist_customers_dataset.csv")
order_items = pd.read_csv(local+"olist_order_items_dataset.csv")
order_payments = pd.read_csv(local+"olist_order_payments_dataset.csv")
orders = pd.read_csv(local+"olist_orders_dataset.csv")
products = pd.read_csv(local+"olist_products_dataset.csv")
sellers = pd.read_csv(local+"olist_sellers_dataset.csv")
reviews = pd.read_csv(local+"olist_order_reviews_dataset.csv")

In [19]:
customers.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [20]:
order_items.head(2)

,order_id,order_item_id,product_id,seller_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.9,19.93


In [21]:
order_payments.head(2)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39


In [22]:
orders.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00


In [23]:
products.head(2)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0


In [24]:
sellers.head(2)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP


In [29]:
reviews.head(2)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13


## Análises Univariadas [Thais]

In [54]:
dfs = {"customers": customers, "sellers": sellers, "products": products, "orders": orders, "order_items": order_items, "order_payments": order_payments, "reviews": reviews}

In [55]:
# verificando a existência de linhas nulas
for names, tables in dfs.items():
    print(f"{names}\n-----linhas nulas:\n{dfs[names].isna().sum()}\n\n-----tipo de dado:\n{dfs[names].dtypes}\n---------------\n\n")

customers
-----linhas nulas:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

-----tipo de dado:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object
---------------


sellers
-----linhas nulas:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

-----tipo de dado:
seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object
---------------


products
-----linhas nulas:
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_lengt

In [56]:
# deletando linhas nulas
for names, tables in dfs.items():
    print(f"{names}\n-----deletando linhas\n")
    dfs[names].dropna(axis=0, inplace=True)

customers
-----deletando linhas

sellers
-----deletando linhas

products
-----deletando linhas

orders
-----deletando linhas

order_items
-----deletando linhas

order_payments
-----deletando linhas

reviews
-----deletando linhas



In [46]:
customers[["customer_city", "customer_state"]].describe()

,customer_city,customer_state
count,99441,99441
unique,4119,27
top,sao paulo,SP
freq,15540,41746


In [50]:
sellers[["seller_city", "seller_state"]].describe()

,seller_city,seller_state
count,3095,3095
unique,611,23
top,sao paulo,SP
freq,694,1849


In [62]:
products[["product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"]].describe()

,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32340.000000,32340.000000,32340.000000,32340.000000
mean,2276.956586,30.854545,16.958813,23.208596
std,4279.291845,16.955965,13.636115,12.078762
min,0.000000,7.000000,2.000000,6.000000
25%,300.000000,18.000000,8.000000,15.000000
50%,700.000000,25.000000,13.000000,20.000000
75%,1900.000000,38.000000,21.000000,30.000000
max,40425.000000,105.000000,105.000000,118.000000


In [64]:
products[["product_category_name"]].describe()

,product_category_name
count,32340
unique,73
top,cama_mesa_banho
freq,3029


In [66]:
orders[["order_status"]].describe()

,order_status
count,96461
unique,2
top,delivered
freq,96455


In [48]:
order_items[["price", "freight_value"]].describe()

,price,freight_value
count,112650.000000,112650.000000
mean,120.653739,19.990320
std,183.633928,15.806405
min,0.850000,0.000000
25%,39.900000,13.080000
50%,74.990000,16.260000
75%,134.900000,21.150000
max,6735.000000,409.680000


In [70]:
order_payments[["payment_type"]].describe()

,payment_type
count,103886
unique,5
top,credit_card
freq,76795


In [71]:
order_payments[["payment_value"]].describe()

,payment_value
count,103886.000000
mean,154.100380
std,217.494064
min,0.000000
25%,56.790000
50%,100.000000
75%,171.837500
max,13664.080000


## Análises Multivariadas